### Basic Usage

This is how to generate short \<svg\> snippets with basic elements:

In [1]:
from svg_snip.Composer import Composer
from svg_snip.Elements import circle
svg = Composer([200,200])
svg.add(circle, cx=100, cy=100, r=10, stroke='blue')
print(svg.render())

<svg width="200" height="200" viewBox="0 0 200 200" xmlns="http://www.w3.org/2000/svg">
  <circle cx="100.00" cy="100.00" r="10.00" stroke="blue" />
</svg>


Please see (generated) documentation for supported arguments.
Unknown arguments will not become attributes in generated code.

In [2]:
from svg_snip.Elements import line
line?

Signature: line(**kwargs)
Docstring:
Generate SVG code for the <line> element.

Parameters:
- x1: (float): The x-coordinate of the start point of the line.
- y1: (float): The y-coordinate of the start point of the line.
- x2: (float): The x-coordinate of the end point of the line.
- y2: (float): The y-coordinate of the end point of the line.
- stroke: (str): The stroke color of the outline. Examples: 'currentColor' keyword, 'red', #FF0000, rgb(255,0,0) ...
- stroke_width: (float): The width of the outline.
- stroke_opacity: (float): The opacity of the outline. A value between 0 (transparent) and 1 (opaque).
- stroke_dasharray: (str): Defines the pattern of dashes and gaps used in the outline.
- stroke_linecap: (str): Specifies the shape to be used at the end of the outline. Possible values are 'butt', 'round', and 'square'.
- stroke_linejoin: (str): Specifies the shape to be used at the corners of the outline. Possible values are 'miter', 'round', and 'bevel'.
- stroke_miterlimit: (flo

### Custom/complex shapes

Simply build a Python function that returns svg code snippets and use them with the Composer class.

You can also add definitions which will be inlcuded only once per generated code. Simply check out how the heart is implemented in this packages's source code.


In [3]:
from svg_snip.Elements import heart
heart?

Signature: heart(x=0, y=0, size=4, angle=0, fill='', **kwargs)
Docstring:
Generate SVG code for a heart.

Args:
    x, y: position
    size: size of the heart.
File:      ~/work/2024_epipolar_transfer_proba/svg_snip/Elements.py
Type:      function


### Usage in Jupyter

To see what the generated SVG looks like you can use IPython.core.display.

This package includes for utility a display() function.
`

In [4]:
from svg_snip.Composer import Composer
from svg_snip.Elements import heart
svg = Composer([200,200])
svg.add(heart, x=75, y=75, size=50, fill='red')
svg.display()

In [5]:
rect?

Object `rect` not found.


### Combining shapes

Just call `Composer.add` multiple times to arrange and style multiple elements.

Pro tip: debug=True to the display call for "show html" details section.

In [6]:
from svg_snip.Composer import Composer
from svg_snip.Elements import heart, rect
svg = Composer([250,200])
svg.add(rect, x=0, y=25, width=250, height= 50, fill='blue')
svg.add(rect, x=0, y=125, width=250, height= 50, fill='blue')
svg.add(heart, x=10, y=100, size=100, angle=-45, fill='red')

svg.display(debug=True)

### Extending the functionality

In [7]:
def x(x=0, y=0, **kwargs):
    return f"""\
<g>
  <line x1="{x-5}" y1="{y-5}" x2="{x+5}" y2="{y+5}" stroke="green"/>
  <line x1="{x-5}" y1="{y+5}" x2="{x+5}" y2="{y-5}" stroke="green"/>
</g>"""

from svg_snip.Composer import Composer
from svg_snip.Elements import rect

svg = Composer((100,100))
svg.add(rect, x=0, y=0, width=100, height=100, fill='white')
svg.add(x, x=50, y=50)
svg.display(debug=True)

In [8]:
from svg_snip.Composer import Composer
from svg_snip.Elements import rect, line

def x(x=0, y=0, **kwargs):
    elements = ['<g>',
                line(x1=x-5, y1=y-5, x2=x+5, y2=y+5, **kwargs),
                line(x1=x-5, y1=y+5, x2=x+5, y2=y-5, **kwargs)]
    return '\n  '.join(elements) + '\n</g>'

svg = Composer((100,100))
svg.add(rect, x=0, y=0, width=100, height=100, fill='white')
svg.add(x, x=50, y=50, stroke='blue', stroke_width=2)
svg.display(debug=True)

### Using definitions

Note in the generated SVG code, you'll find a single definition for `red_x` that is then `<use>`d three times. Also note how the last `x` has a different stroke width.

In [9]:
from svg_snip.Elements import attributes, stroke_attributes

def red_x(x=0, y=0, size=6, **kwargs):
    attrib = attributes(stroke_attributes, **kwargs)
    return f'<use {attrib} xlink:href="#red_x" transform="translate({x:.2f},{y:.2f}) scale({size / 6:.2f})"/>'

Composer.declare(red_x, {'red_x': '\n  '.join(['<g id="red_x">',
                   line(x1=-5, y1=-5, x2=+5, y2=+5, stroke='red'),
                   line(x1=-5, y1=+5, x2=+5, y2=-5, stroke='red')
                  ]) + '\n</g>'})

svg = Composer((100,100))
svg.add(rect, x=0, y=0, width=100, height=100, fill='white')
svg.add(red_x, x=50, y=50)
svg.add(red_x, x=25, y=25, stroke_width=2)
svg.add(red_x, x=75, y=25, transform="rotate(30)")
svg.display(debug=True)

### Go crazy

Once you start visualizing, the next step is usually interaction.

For this purpose we provide a usefull class in CanvasWithOverlay built on top of ipycanvas. Here's simple usage (without SVG).


In [10]:
from svg_snip.Jupyter import CanvasWithOverlay

vis = CanvasWithOverlay(200,200)

def handle_draw(vis):
    x,y = vis.mouse_state.pos()
    vis.canvas[1].clear()
    vis.canvas[1].fill_style = "red" if vis.mouse_state.clicked else "blue"
    vis.canvas[1].fill_rect(x-2,y-2,5,5)
    if vis.mouse_state.clicked:
        vis.canvas[0].fill_style = "#00000011"
        vis.canvas[0].fill_rect(x-2,y-2,5,5)
vis.handle_draw = handle_draw

vis.display()


How here's how to include a real SVG overlay to the Canvas

In [11]:
from svg_snip.Jupyter import CanvasWithOverlay

from svg_snip.Composer import Composer
from svg_snip.Elements import star

vis = CanvasWithOverlay(200,200)

def handle_draw(vis):
    x,y = vis.mouse_state.pos()
    svg = Composer((vis.w, vis.h))
    svg.add(star, x=x, y=y, size=8,
        fill="red" if vis.mouse_state.clicked else "blue")
    vis.html_overlay.value = svg.render()
    
vis.handle_draw = handle_draw

vis.display()